<a href="https://colab.research.google.com/github/Valgueiro/dataScience_project/blob/master/Projeto_1_dataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisandos os dados da temporada 2019/20 da NFL

Utilizarei os dados de https://www.kaggle.com/dvc415/nfl-2019-weekly-player-stats . Nele podemos encontrar estatísticas sobre diversos jogadores de 3 posiçoes do futebol americano:

* Quaterbacks (qbs) - A mente por trás do ataque, são os responsáveis por liderar e coordenar o ataque

* Running Backs (rbs) - corredores da equipe, geralmente recebem a bola diretamente do Quarteback e correm para dentro da defesa.

* Wide receivers (wrs) - também são corredores, mas geralmente correm pra dentro da defesa adversária e recebem a bola do QB através de um passe.

## Pré-processando os dados

### Coletando dados



In [2]:
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
qbs = pd.read_csv("https://raw.githubusercontent.com/Valgueiro/dataScience_project/master/data/nfl_2019_qb_weekly_stats.csv")

### Filtrando e reorganizando os dados de Qbs

In [4]:
qbs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           611 non-null    int64  
 1   Player         611 non-null    object 
 2   Position       611 non-null    object 
 3   Age            611 non-null    float64
 4   Date           611 non-null    object 
 5   League         611 non-null    object 
 6   Team           611 non-null    object 
 7   Opp            611 non-null    object 
 8   Result         611 non-null    object 
 9   Game           611 non-null    int64  
 10  Week           611 non-null    int64  
 11  Day            611 non-null    object 
 12  Cmp            611 non-null    int64  
 13  Pass_Att       611 non-null    int64  
 14  Cmp_Per        611 non-null    float64
 15  Pass_Yards     611 non-null    int64  
 16  Pass_TD        611 non-null    int64  
 17  Pass_Int       611 non-null    int64  
 18  QB_Rate   

Logo de cara podemos ver que não temos dados ausentes, mas alguns tipos devem ser estudados e melhorados, assim como algumas colunas devem ser discretizadas.

#### Verificando a necessidade da coluna "Position"

In [5]:
qbs["Position"].unique()

array(['QB', 'WR', 'RB', 'K', 'P', 'FB/TE'], dtype=object)

In [6]:
qbs[qbs["Position"] == 'WR'][["Player", "Pass_Att"]]

,Player,Pass_Att
14,Julian Edelman,1
43,Dante Pettis,1
142,Odell Beckham Jr.,1
146,DeAndre Hopkins,1
151,Odell Beckham Jr.,1
248,Zach Pascal,1
338,Randall Cobb,1
343,Julian Edelman,1
348,Courtland Sutton,1
411,Steven Sims,1


Como podemos ver, o dataset conta também com jogadores de outra posição que jogaram de QB em algum momento, como por exemplo os Wide Receivers que vemos nesta ultima tabela. Com as duas colunas mostradas podemos ver que todos eles fizeram apenas 2 passes no máximo. Geralmente essas jogadas são fora do comum, são chamadas de "trick play".

#### Verificando o tipo da coluna "Age"

In [7]:
qbs["Age"].describe()

count    611.000000
mean      28.154298
std        5.364599
min       22.032000
25%       24.046500
50%       26.308000
75%       31.219000
max       42.148000
Name: Age, dtype: float64

In [8]:
qbs[qbs["Player"] == "Kirk Cousins"][["Player","Age","Date"]]

,Player,Age,Date
4,Kirk Cousins,31.020,2019-09-08
72,Kirk Cousins,31.027,2019-09-15
86,Kirk Cousins,31.034,2019-09-22
126,Kirk Cousins,31.041,2019-09-29
149,Kirk Cousins,31.048,2019-10-06
186,Kirk Cousins,31.055,2019-10-13
219,Kirk Cousins,31.062,2019-10-20
259,Kirk Cousins,31.066,2019-10-24
298,Kirk Cousins,31.076,2019-11-03
318,Kirk Cousins,31.083,2019-11-10


Como podemos ver, apesar de idade normalmente ser tratada com inteiro, aqui é mostrada a idade inclusive incluindo a quantidade de meses e dias.

#### Transformando o tipo da coluna Date

Com o describe, vimos que a coluna Date está com o tipo Object. Devemos mudá-la para o tipo correto de datas.

In [9]:
qbs["Date"].head()

0    2019-09-08
1    2019-09-08
2    2019-09-08
3    2019-09-08
4    2019-09-08
Name: Date, dtype: object

In [10]:
qbs["Date"] = pd.to_datetime(qbs["Date"])
qbs["Date"].head()

0   2019-09-08
1   2019-09-08
2   2019-09-08
3   2019-09-08
4   2019-09-08
Name: Date, dtype: datetime64[ns]

#### Verificando a necessidade da coluna League

In [11]:
qbs["League"].unique()

array(['NFL'], dtype=object)

Como podemos ver todas as linhas tem o mesmo valor para esta coluna. Como já é sabido que estamos no contexto da NFL, vou retirá-la do dataset.

In [12]:
qbs.drop(columns=['League'])

,Rank,Player,Position,Age,Date,Team,Opp,Result,Game,Week,Day,Cmp,Pass_Att,Cmp_Per,Pass_Yards,Pass_TD,Pass_Int,QB_Rate,Sacked,Yds_Lost,Yards_Att,Adj_Yards_Att
0,1,Lamar Jackson,QB,22.244,2019-09-08,BAL,MIA,W 59-10,1,1,Sun,17,20,85.00,324,5,0,158.3,1,1,16.20,16.20
1,2,Dak Prescott,QB,26.041,2019-09-08,DAL,NYG,W 35-17,1,1,Sun,25,32,78.13,405,4,0,158.3,0,0,12.66,12.66
2,3,Robert Griffin III,QB,29.208,2019-09-08,BAL,MIA,W 59-10,1,1,Sun,6,6,100.00,55,1,0,144.4,0,0,9.17,9.17
3,4,Patrick Mahomes,QB,23.356,2019-09-08,KAN,JAX,W 40-26,1,1,Sun,25,33,75.76,378,3,0,143.2,0,0,11.45,11.45
4,5,Kirk Cousins,QB,31.020,2019-09-08,MIN,ATL,W 28-12,1,1,Sun,8,10,80.00,98,1,0,140.8,1,1,9.80,9.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,36,Robert Griffin III,QB,29.320,2019-12-29,BAL,PIT,W 28-10,16,17,Sun,11,21,52.38,96,0,1,44.9,3,15,4.57,4.57
607,37,David Blough,QB,24.151,2019-12-29,DET,GNB,L 20-23,16,17,Sun,12,29,41.38,122,0,1,39.7,1,7,4.21,4.21
608,38,Matt Haack,P,25.187,2019-12-29,MIA,NWE,W 27-24,16,17,Sun,0,1,0.00,0,0,0,39.6,0,0,0.00,0.00
609,39,Sean Mannion,QB,27.248,2019-12-29,MIN,CHI,L 19-21,16,17,Sun,12,21,57.14,126,0,2,35.1,0,0,6.00,6.00


#### Separar dados da coluna Result

In [13]:
qbs["Result"].head()

0    W 59-10
1    W 35-17
2    W 59-10
3    W 40-26
4    W 28-12
Name: Result, dtype: object

A coluna result contem 3 informaçoes: O resultado final do jogo para o jogador, a quantidade de pontos feitos e a quantidade de pontos recebidos. Vamos separar esses dados em 3 colunas distintas

In [14]:
import re
parsed_results = qbs["Result"].map(lambda result: re.findall('(.+) (\d+)-(\d+)', result)[0])
qbs['Game_result'] = parsed_results.map(lambda parsed: parsed[0])
qbs['Points_scored'] = parsed_results.map(lambda parsed: parsed[1])
qbs['Points_Taken'] = parsed_results.map(lambda parsed: parsed[2])

qbs[['Game_result', 'Points_scored', 'Points_Taken']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Game_result    611 non-null    object
 1   Points_scored  611 non-null    object
 2   Points_Taken   611 non-null    object
dtypes: object(3)
memory usage: 14.4+ KB


Após dividir as informaçoes, vamos mudar o tipo para inteiro e apagar a coluna que foi dividida.

In [15]:
qbs['Points_scored'] = pd.to_numeric(qbs['Points_scored'])
qbs['Points_Taken'] = pd.to_numeric(qbs['Points_Taken'])
qbs[['Game_result', 'Points_scored', 'Points_Taken']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Game_result    611 non-null    object
 1   Points_scored  611 non-null    int64 
 2   Points_Taken   611 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 14.4+ KB


In [16]:
qbs = qbs.drop(columns=['Result'])

### Resultado final


In [17]:
qbs.head()

,Rank,Player,Position,Age,Date,League,Team,Opp,Game,Week,Day,Cmp,Pass_Att,Cmp_Per,Pass_Yards,Pass_TD,Pass_Int,QB_Rate,Sacked,Yds_Lost,Yards_Att,Adj_Yards_Att,Game_result,Points_scored,Points_Taken
0,1,Lamar Jackson,QB,22.244,2019-09-08,NFL,BAL,MIA,1,1,Sun,17,20,85.00,324,5,0,158.3,1,1,16.20,16.20,W,59,10
1,2,Dak Prescott,QB,26.041,2019-09-08,NFL,DAL,NYG,1,1,Sun,25,32,78.13,405,4,0,158.3,0,0,12.66,12.66,W,35,17
2,3,Robert Griffin III,QB,29.208,2019-09-08,NFL,BAL,MIA,1,1,Sun,6,6,100.00,55,1,0,144.4,0,0,9.17,9.17,W,59,10
3,4,Patrick Mahomes,QB,23.356,2019-09-08,NFL,KAN,JAX,1,1,Sun,25,33,75.76,378,3,0,143.2,0,0,11.45,11.45,W,40,26
4,5,Kirk Cousins,QB,31.020,2019-09-08,NFL,MIN,ATL,1,1,Sun,8,10,80.00,98,1,0,140.8,1,1,9.80,9.80,W,28,12


In [18]:
qbs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Rank           611 non-null    int64         
 1   Player         611 non-null    object        
 2   Position       611 non-null    object        
 3   Age            611 non-null    float64       
 4   Date           611 non-null    datetime64[ns]
 5   League         611 non-null    object        
 6   Team           611 non-null    object        
 7   Opp            611 non-null    object        
 8   Game           611 non-null    int64         
 9   Week           611 non-null    int64         
 10  Day            611 non-null    object        
 11  Cmp            611 non-null    int64         
 12  Pass_Att       611 non-null    int64         
 13  Cmp_Per        611 non-null    float64       
 14  Pass_Yards     611 non-null    int64         
 15  Pass_TD        611 non-

## Analisando os dados

### Os mais bem rankeados

Primeiramente vamos ver quais foram os qbs mais bem rankeados na temporada. Para isso, vamos agrupar os dados por jogador.

In [19]:
all_players_qb_stats = qbs.groupby(['Player', 'Position'])
all_players_qb_stats.head()

,Rank,Player,Position,Age,Date,League,Team,Opp,Game,Week,Day,Cmp,Pass_Att,Cmp_Per,Pass_Yards,Pass_TD,Pass_Int,QB_Rate,Sacked,Yds_Lost,Yards_Att,Adj_Yards_Att,Game_result,Points_scored,Points_Taken
0,1,Lamar Jackson,QB,22.244,2019-09-08,NFL,BAL,MIA,1,1,Sun,17,20,85.00,324,5,0,158.3,1,1,16.20,16.20,W,59,10
1,2,Dak Prescott,QB,26.041,2019-09-08,NFL,DAL,NYG,1,1,Sun,25,32,78.13,405,4,0,158.3,0,0,12.66,12.66,W,35,17
2,3,Robert Griffin III,QB,29.208,2019-09-08,NFL,BAL,MIA,1,1,Sun,6,6,100.00,55,1,0,144.4,0,0,9.17,9.17,W,59,10
3,4,Patrick Mahomes,QB,23.356,2019-09-08,NFL,KAN,JAX,1,1,Sun,25,33,75.76,378,3,0,143.2,0,0,11.45,11.45,W,40,26
4,5,Kirk Cousins,QB,31.020,2019-09-08,NFL,MIN,ATL,1,1,Sun,8,10,80.00,98,1,0,140.8,1,1,9.80,9.80,W,28,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,34,Matt Barkley,QB,29.112,2019-12-29,NFL,BUF,NYJ,16,17,Sun,18,35,51.43,232,0,2,48.8,1,1,6.63,6.63,L,6,13
607,37,David Blough,QB,24.151,2019-12-29,NFL,DET,GNB,16,17,Sun,12,29,41.38,122,0,1,39.7,1,7,4.21,4.21,L,20,23
608,38,Matt Haack,P,25.187,2019-12-29,NFL,MIA,NWE,16,17,Sun,0,1,0.00,0,0,0,39.6,0,0,0.00,0.00,W,27,24
609,39,Sean Mannion,QB,27.248,2019-12-29,NFL,MIN,CHI,16,17,Sun,12,21,57.14,126,0,2,35.1,0,0,6.00,6.00,L,19,21


Primeiramente, podemos observar a coluna de QB_Rate, que trás o Passer Rating do jogador em cada uma das partidas que ele jogou. 

O cáculo deste número é definido pela liga e leva em comparaçao atributos como o número de passes tentados, de passes completos, touchdowns, etc. O interessante desta métrica é que seu valor máximo é de **158,3**, e seu valor mínimo, **0**.

Para mais informações sobre como é feito seu cálculo https://pt.wikipedia.org/wiki/Passer_rating


In [20]:
all_players_qb_stats['Player', 'Position', 'QB_Rate'].mean().sort_values(['QB_Rate'],ascending=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,,QB_Rate
Player,Position,
Danny Amendola,WR,158.30
Emmanuel Sanders,WR,158.30
John Brown,WR,158.30
Julian Edelman,WR,138.50
Matt Schaub,QB,120.70
...,...,...
Jarrett Stidham,QB,38.55
Sean Mannion,QB,35.10
Will Grier,QB,23.00


Como podemos ver nos valores maiores e menores mostrados, todos os jogadores com o rate máximo são de posições diferentes de QB. Isso se deve marjoritariamente ao fato de que jogadores de outras posições podem até jogar como QB em jogadas "engraçadinhas", mas essas são poucas durante uma temporada inteira. Entao, vamos retirar esses outros jogadores.

In [21]:
qb_players = qbs[qbs['Position'] == 'QB'][['Player', 'QB_Rate','Team']].groupby(['Player','Team']).agg( 
                                                  QB_Rate_Mean=pd.NamedAgg(column='QB_Rate', aggfunc='mean'),
                                                  QB_Rate_Median=pd.NamedAgg(column='QB_Rate', aggfunc='median'),
                                                  Games_Played=pd.NamedAgg(column='QB_Rate', aggfunc='count'))
qb_players.head(10)

,,QB_Rate_Mean,QB_Rate_Median,Games_Played
Player,Team,,,
A.J. McCarron,HOU,52.400000,52.40,2
Aaron Rodgers,GNB,95.681250,90.70,16
Alex Tanney,NYG,79.200000,79.20,1
Andy Dalton,CIN,76.753846,75.00,13
Baker Mayfield,CLE,76.881250,81.60,16
Ben Roethlisberger,PIT,66.500000,66.50,2
Blake Bortles,LAR,56.200000,56.20,1
Brandon Allen,DEN,73.300000,61.90,3
Brett Hundley,ARI,60.200000,61.80,3


Agora vamos analisar os ratings apenas dos Quartebacks presentes no dataset

In [22]:
sorted_by_rate = qb_players.sort_values(['QB_Rate_Mean'],ascending=False)
sorted_by_rate.head(10)

,,QB_Rate_Mean,QB_Rate_Median,Games_Played
Player,Team,,,
Matt Schaub,ATL,120.700000,110.90,3
Ryan Tannehill,TEN,118.927273,130.80,11
Drew Brees,NOR,109.872727,116.40,11
Lamar Jackson,BAL,108.460000,104.80,15
Kirk Cousins,MIN,108.066667,111.50,15
Russell Wilson,SEA,107.793750,108.45,16
Matthew Stafford,DET,105.100000,110.20,8
Patrick Mahomes,KAN,104.842857,104.30,14
Mike Glennon,OAK,104.700000,104.70,2


Vemos outro ponto estranho: **Matt Schaub** é o Quarterback Reserva de Atlanta e jogou apenas 3 jogos na temporada 2019, e - por ter jogado poucos jogos - sua média está bem alta. Para evitar casos como esses, vamos retirar quartebacks que jogaram menos do que 5 jogos na temporada, ou seja, jogaram mais de 30% da temporada.

In [23]:
qb_players_with_at_least_5_games = qb_players[qb_players['Games_Played'] > 5]
qb_players_with_at_least_5_games.head(10)

,,QB_Rate_Mean,QB_Rate_Median,Games_Played
Player,Team,,,
Aaron Rodgers,GNB,95.681250,90.70,16
Andy Dalton,CIN,76.753846,75.00,13
Baker Mayfield,CLE,76.881250,81.60,16
Carson Wentz,PHI,93.668750,94.00,16
Case Keenum,WAS,87.155556,98.40,9
Dak Prescott,DAL,100.956250,96.15,16
Daniel Jones,NYG,85.961538,78.60,13
Derek Carr,OAK,100.031250,104.55,16
Deshaun Watson,HOU,98.393333,92.10,15


Agora vamos ver de fato quais foram os melhores quartebacks da temporada baseado no QB Rating médio.

In [24]:
sorted_qb_players_with_at_least_5_games = qb_players_with_at_least_5_games.sort_values(['QB_Rate_Mean'],ascending=False)
sorted_qb_players_with_at_least_5_games.head(10)

,,QB_Rate_Mean,QB_Rate_Median,Games_Played
Player,Team,,,
Ryan Tannehill,TEN,118.927273,130.80,11
Drew Brees,NOR,109.872727,116.40,11
Lamar Jackson,BAL,108.460000,104.80,15
Kirk Cousins,MIN,108.066667,111.50,15
Russell Wilson,SEA,107.793750,108.45,16
Matthew Stafford,DET,105.100000,110.20,8
Patrick Mahomes,KAN,104.842857,104.30,14
Jimmy Garoppolo,SFO,102.512500,109.35,16
Teddy Bridgewater,NOR,101.328571,100.90,7


Com o conhecimento de como foi a temporada 2019, podemos perceber que, tirando Matthew Stafford do Lions e Dak Prescott do Dallas cowboys, todos os Qbs presentes nessa lista estavam nos playoffs. Isto é, fazem parte dos 12 melhores times da temporada.

Vamos verificar se o resultado seria distinto caso tivessemos usado a mediana ao invés da média.

In [26]:
qb_players_with_at_least_5_games.sort_values(['QB_Rate_Median'],ascending=False).head(10)

,,QB_Rate_Mean,QB_Rate_Median,Games_Played
Player,Team,,,
Ryan Tannehill,TEN,118.927273,130.80,11
Drew Brees,NOR,109.872727,116.40,11
Kirk Cousins,MIN,108.066667,111.50,15
Matthew Stafford,DET,105.100000,110.20,8
Jimmy Garoppolo,SFO,102.512500,109.35,16
Russell Wilson,SEA,107.793750,108.45,16
Lamar Jackson,BAL,108.460000,104.80,15
Derek Carr,OAK,100.031250,104.55,16
Patrick Mahomes,KAN,104.842857,104.30,14


Aqui percebemos algumas diferenças. Alguns nomes novos apareceram (como Derek Carr), e outros desapareceram (como Dak Prescott). De qualquer forma, o Top 2 se manteve inalterado. 

Apesar da diferença, resolvi manter utilizando a média visto que os "outliers" neste conjunto de dados definem como pontos altos ou baixos do jogador em partidas, e não "pontos fora da curva". Jogos onde o QB jogou mal devem ser levados em conta para saber se de fato o jogador é consistente.

E os quaterbacks com os piores ratings médios:

In [27]:
sorted_qb_players_with_at_least_5_games.tail(10)

,,QB_Rate_Mean,QB_Rate_Median,Games_Played
Player,Team,,,
Daniel Jones,NYG,85.961538,78.6,13
Joe Flacco,DEN,85.112500,81.4,8
Mitchell Trubisky,CHI,83.953333,70.1,15
Mason Rudolph,PIT,83.300000,85.0,10
Kyle Allen,CAR,82.400000,84.2,13
Devlin Hodges,PIT,80.375000,91.8,8
Dwayne Haskins,WAS,77.266667,74.6,9
Baker Mayfield,CLE,76.881250,81.6,16
Andy Dalton,CIN,76.753846,75.0,13


### Comparaçao entre Tannehill e Mahomes

Vamos agora comparar o rating do Quaterback com maior média (Ryan Tannehill) e o MVP da temporada e ganhador do superbowl (Patrick Mahomes).


In [28]:
tannehill_stats = qbs[(qbs['Player'] == 'Ryan Tannehill') | (qbs['Player'] == 'Patrick Mahomes')][['Player','QB_Rate', 'Week']].sort_values(['Week'],ascending=True)

import plotly.express as px

fig = px.line(tannehill_stats, x="Week", y="QB_Rate", title='Comparaçao entre Ryan Tannehill e Patrick Mahomes', color='Player', labels={
    "QB_Rate": "Quarterback Rating"
} )
fig.show()

Como podemos ver no gráfico, Tannehill só começou a jogar a partir da semana 6, mas mesmo assim apresentou número impressionantes principalmente após a semana 10, enquanto o MVP teve uma queda de rendimento.

### Verificando se há alguma relaçao entre o número de passes áreaos e o número de sacks



**[Sacks](https://pt.wikipedia.org/wiki/Quarterback_sack#:~:text=No%20futebol%20americano%20dos%20Estados,de%20scrimmage%20dentro%20do%20%22pocket%22)**  são muito famosos no futebol americano. Eles ocorrem quando um quarterback é atingido por um defensor antes de "soltar" a bola.

No futebol americano há, marjoritariamente, dois tipos de jogadas: áreas de terrestres. Enquanto na última geralmente o QB libera a bola logo após o ínicio da jogada, as primeiras dependem ainda de uma movimentaçao e análise do mesmo durante alguns segundos antes de lançar a bola. E esses segundos são cruciais para o Quarteback, pois, enquanto ele está decidindo o que vai fazer, a linha ofensiva briga contra a linha defensiva para que estes não chegem até o QB, acontecendo assim o famoso **sack**. 

Assim, times que corram mais com a bola provavelmente sofrerão menos sacks. Já quando há uma grande quantidade de jogadas áreas, o número de sacks tende a ser maior. 

Queremos verificar essa correlação utilizando os dados que temos.




Nos dados a coluna Sacked fala quantas vezes o QB sofreu Sack durante o jogo 

In [29]:
qbs['Sacked'].unique()

array([1, 0, 4, 2, 6, 3, 5, 9, 8, 7])

Já a coluna Pass_Att fala quantas vezes o QB tentou uma jogada área (também chamada de jogada de passe) no jogo

In [30]:
qbs['Pass_Att'].unique()

array([20, 32,  6, 33, 10, 24,  8, 36, 25, 34, 26, 39, 27,  1, 44, 30, 45,
       51, 43, 31, 46, 41,  4, 54, 38, 37, 29, 47,  3, 35, 28, 19, 42, 40,
       50,  5, 15, 18, 21,  2, 22, 53, 68, 16, 17, 11, 23,  9, 49, 48, 12,
       52,  7, 14, 56, 55])

Testando a correlação das duas variáveis através do metodo spearman.

In [89]:
# qbs[['Pass_Att', 'Sacked']].corr(method='pearson')
stats_by_week.corr(method='pearson')

,Norm_Sacked,Norm_Pass_Att
Norm_Sacked,1.000000,0.394492
Norm_Pass_Att,0.394492,1.000000


Apesar de na teoria ter sentido as duas serem correlatas, na prática não há uma correlção forte entre as duas variáveis.



Agora vamos ver um gráfico mostrando a relaçao entre o número de sacks e a tentaivas de jogadas de passe em cada uma das semanas.

In [97]:
summed_variables = qbs.groupby(['Week'])[['Sacked', 'Pass_Att']].agg('sum').reset_index();

import plotly.graph_objects as go

# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=summed_variables['Week'], y=summed_variables['Pass_Att'],
                    mode='lines+markers',
                    name='Tentativas de passe'))

fig.add_trace(go.Scatter(x=summed_variables['Week'], y=summed_variables['Sacked'],
                    mode='lines+markers',
                    name='Numero de Scaks'))

fig.update_layout(title='Soma do número de sacks e de tentativas de passe por semana',
                   xaxis_title='Semana')
fig.show()

### Comparar quantidade de jogadas de passe entre Qbs vencedores e perdedores

Agora, quero fazer uma pesquisa a respeito da "efetividade" de jogadas de passe na temparada 2019/20. Aqui vamos fazer um estudo para saber se há uma diferença estátistica no número de passes tentados entre times que ganharam e que perderam.

Separando os dados dos perdedores e ganhadores

In [100]:
loser_player_data = qbs[qbs['Game_result'] == 'L']
winner_player_data = qbs[qbs['Game_result'] == 'W']

Testaremos a normalidade dos dois grupos utilizando o teste **Shapiro-Wilk**  Nele a Hipótese nula é que a população é normalmente distribuída. 




In [101]:
stats.shapiro(loser_player_data['Pass_Att'])[1]

7.482242256329208e-12

In [102]:
stats.shapiro(winner_player_data['Pass_Att'])[1]

3.203032186271243e-13

Como p-value é muito baixo nas duas amostras, ambas nao sao normalmente distribuidas. Como sabemos que os dois grupos não são pareados, vamos utilizar o método de Mann-Whitney. Nele temos que a hipoótese nula é que as amostras vêm de populações com a mesma distribuição

In [44]:
stats.mannwhitneyu(loser_player_data['Pass_Att'], winner_player_data['Pass_Att'])

MannwhitneyuResult(statistic=35434.5, pvalue=2.4264846662494796e-07)

Como o p-value é muito baixo, as amostras vêm de distribuições diferentes e, então, há uma diferença estatística entre o número de passes tentados por equipes que ganharam e perderam a partida.